In [1]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Step 1: Authenticate with Google Sheets
# Define the scope and path to your service account key file
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/tawate/Documents/Dinner Club/clean-terminal-468616-k8-06169f567f7e.json', scope)
client = gspread.authorize(creds)

# Step 2: Open the spreadsheet
# You can open it by title, key, or URL
try:
    # Open by title
    sheet = client.open("Dinner Club Ranking (Responses)").sheet1  # Use .sheet1 for the first worksheet

    # Or, open by spreadsheet key (the long string in the URL)
    # sheet = client.open_by_key("your-spreadsheet-key").sheet1

    # Or, open by URL
    # sheet_url = "https://docs.google.com/spreadsheets/d/your-spreadsheet-key/edit"
    # sheet = client.open_by_url(sheet_url).sheet1

except gspread.exceptions.SpreadsheetNotFound:
    print("Spreadsheet not found. Please check the title and sharing permissions.")
    exit()

# Step 3: Get all the data from the worksheet
data = sheet.get_all_values()

# Step 4: Convert the data into a pandas DataFrame
# The first row is typically the header
headers = data[0]
df = pd.DataFrame(data[1:], columns=headers)

# Display the DataFrame
print(df.head())

# You can now work with the pandas DataFrame as you normally would
# For example, to print a specific column:
# print(df['Column Name'])

            Timestamp Respondent Name             Restaurant  \
0  8/10/2025 13:31:23          Taylor              Mala Pata   
1  8/10/2025 14:22:56          Taylor  Naos Hellenic Cuisine   
2  8/10/2025 19:26:17           Emily              Mala Pata   
3  8/10/2025 19:26:41           Emily  Naos Hellenic Cuisine   

  Food Quality (0 - 10) Ambience  Bathroom Quality Food Portion Size (0 - 10)  \
0                     7         5                3                          7   
1                     8         9                8                          8   
2                     8         4                5                          5   
3                     9        10               10                          8   

  Service (0 - 10) Notes (Optional) Drinks (0 - 10) Column 8  
0                8                                4           
1                7                                7           
2                8                                9           
3                8   

In [27]:
# Apply Weights to each category
restaurant_rankings = {
    'Food Taste': {
        'weight': 0.5
    },
    'Food Portion Size': {
        'weight': 0.1
    },
    'Drinks': {
        'weight': 0.1
    },
    'Service': {
        'weight': 0.1
    },
    'Ambiance': {
        'weight': 0.1
    },
    'Bathroom': {
        'weight': 0.1
    }
} 

# The sum of all weights should ideally be 1.0. Let's check:
total_weight = sum(category['weight'] for category in restaurant_rankings.values())
print(f"Total weight of all categories: {total_weight}\n")

Total weight of all categories: 1.0



In [28]:
# Standardize the column names in the DataFrame to match the keys in restaurant_rankings
df.rename(columns={
    'Food Quality': 'Food Taste',
    'Ambience ': 'Ambiance',
    'Bathroom Quality': 'Bathroom',
    'Food Portion Size (10 = Large, 1 = Small)': 'Food Portion Size',
    'Service': 'Service',
    'Drinks': 'Drinks'
}, inplace=True)

# Convert relevant columns to numeric, handling potential errors
cols_to_convert = ['Food Taste', 'Food Portion Size', 'Service', 'Ambiance', 'Bathroom', 'Drinks']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # 'coerce' will turn invalid parsing into NaN

# Function to calculate weighted ranking
def calculate_weighted_ranking(row):
    weighted_sum = 0
    for category, details in restaurant_rankings.items():
        if category in row.index:  # Check if the category exists in the row
            weighted_sum += row[category] * details['weight']
    return weighted_sum

# Apply the function to each row to create a 'Weighted Ranking' column
df['Weighted Ranking'] = df.apply(calculate_weighted_ranking, axis=1)

# Print the DataFrame with the new 'Weighted Ranking' column
print(df[['Restaurant', 'Food Taste', 'Food Portion Size', 'Drinks', 'Service', 'Ambiance', 'Bathroom', 'Weighted Ranking']].head())

  Restaurant  Food Taste  Food Portion Size  Drinks  Service  Ambiance  \
0  Mala Pata           7                  7       4        8         5   

   Bathroom  Weighted Ranking  
0         3               6.2  


In [29]:
# Calculate average Weighted Ranking by Restaurant
average_rankings = df.groupby('Restaurant')['Weighted Ranking'].mean().sort_values(ascending=False)

# Print the average rankings
print("Average Weighted Ranking by Restaurant:\n", average_rankings)

Average Weighted Ranking by Restaurant:
 Restaurant
Mala Pata    6.2
Name: Weighted Ranking, dtype: float64


In [31]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# Assuming df and restaurant_rankings are already defined as in previous cells

# Get unique restaurants and respondents
restaurants = df['Restaurant'].unique()
respondents = df['Name'].unique()

# Streamlit app layout with sidebar
st.sidebar.header("Filters")

# Add selectbox for restaurant and respondent filters in the sidebar
selected_restaurant = st.sidebar.selectbox('Select Restaurant', ['All'] + list(restaurants))
selected_respondent = st.sidebar.selectbox('Select Respondent', ['All'] + list(respondents))

# Filter the DataFrame based on user selections
filtered_df = df.copy()  # Start with a copy of the original DataFrame

if selected_restaurant != 'All':
    filtered_df = filtered_df[filtered_df['Restaurant'] == selected_restaurant]

if selected_respondent != 'All':
    filtered_df = filtered_df[filtered_df['Name'] == selected_respondent]

# Categories for the spider plot
categories = list(restaurant_rankings.keys())

# Function to create spider plots for each restaurant
def create_spider_plot(df):
    fig = go.Figure()
    
    restaurants_to_plot = df['Restaurant'].unique()  # Use unique restaurants from the filtered DataFrame
    
    for restaurant in restaurants_to_plot:
        restaurant_df = df[df['Restaurant'] == restaurant]
        
        avg_scores = []
        for category in categories:
            if category in restaurant_df.columns:
                avg_scores.append(restaurant_df[category].mean())
            else:
                avg_scores.append(0)
        
        fig.add_trace(go.Scatterpolar(
            r=avg_scores,
            theta=categories,
            fill='toself',
            name=restaurant
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 10]
            )),
        showlegend=True,
        title='Restaurant Ranking Spider Plot'
    )
    return fig

# Main app content
st.title("Restaurant Ranking Spider Plot")

# Create and display the spider plot
fig = create_spider_plot(filtered_df)
st.plotly_chart(fig)

# Calculate average scores per category and overall weighted ranking
average_scores = filtered_df.groupby('Restaurant')[categories].mean()
average_weighted_ranking = filtered_df.groupby('Restaurant')['Weighted Ranking'].mean()
average_scores['Average Weighted Ranking'] = average_weighted_ranking

# Display the average scores in a table
st.subheader("Average Scores per Restaurant")
st.dataframe(average_scores)


2025-08-10 13:38:55.379 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55.491 
  command:

    streamlit run /Users/tawate/Documents/Dinner Club/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-08-10 13:38:55.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:38:55

DeltaGenerator()

In [32]:
import streamlit as st

# Calculate average scores for each category per restaurant
average_scores = df.groupby('Restaurant')[['Food Taste', 'Food Portion Size', 'Drinks', 'Service', 'Ambiance', 'Bathroom']].mean()

# Calculate average weighted ranking per restaurant (already done in CELL INDEX 3, but included here for completeness)
average_weighted_ranking = df.groupby('Restaurant')['Weighted Ranking'].mean()

# Add the average weighted ranking to the average_scores DataFrame
average_scores['Average Weighted Ranking'] = average_weighted_ranking

# Print the average scores
print("Average Scores per Restaurant:\n", average_scores)
# Streamlit app
st.title("Average Scores per Restaurant")

# Display the average scores in a Streamlit table
st.write("Average Scores per Restaurant:")
st.dataframe(average_scores)

2025-08-10 13:45:22.943 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:22.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:22.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:22.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:22.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:22.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:23.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 13:45:23.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Average Scores per Restaurant:
             Food Taste  Food Portion Size  Drinks  Service  Ambiance  \
Restaurant                                                             
Mala Pata          7.0                7.0     4.0      8.0       5.0   

            Bathroom  Average Weighted Ranking  
Restaurant                                      
Mala Pata        3.0                       6.2  


DeltaGenerator()